In [1]:
import numpy as np 
from scipy.stats import norm

class OptionsMath():
    
    """
    Preface:
        The classic Black Scholes formula as implemented below assumes:
            1) The amount of time that is used for the determining the distribution of the underlying value 
            at expiry is the same amount of time that should be used for discounting the forward price of 
            the option.  Frequently these two amounts of time are different as options rarely settle at expiry
            but instead settle at expiry plus a few days.
            2) The interest rate used to project the underlying value at expiry based on the underlying spot 
            value is the same as the interest rate that should be used for discounting the forward price of
            the option.  In crytpo markets in 2025, for example, these two rates are very different.
        To overcome these shortcomings, the following implementation is recommended:
            1) Set the interest rate in the Black Scholes formula to zero.
            2) Calculate the underlying value at expiry prior to calling the formula.  Enter the underlying
            value at epxiry as the undelrying_value variable# Assistant
            3) Calculate the amount of time to expiration of the option prior to calling the formual.  Set
            the time variable equal to this amount.
            4) Discount the formulas's resulting forward price of the option after using the formula below
            using the Time_Value_Money Class and choices for rate and time that are specific to the discounting
            situation (time to payment rather than time to expiry and rate for discounting rather than forward
            rate of underlying).
        The term "value" is used rather than "price" to emphasize that formulas below can be used for rates, too
    
    Parameters:
        c_or_p (string): acceptable names for 'call' or 'put' (see c_or_p function below)
        u (float): see note above, value of underlying
        k (float): strike value expressed in same units as underlying_value
        v (float): volatility expressed as a percentage of underlying value (should be the same value regardless of 
            using forward or spot underlying price)
        t (float): see note above, fraction of a year (use Daycount_Math prior to calling functions below)
        r (float): see note above, continuously compounded rate used to (i) discount forward option 
            value to present value and (ii) create underlying value at expiry (if spot underlying value is used
            as the input) 
        d0 (float): the amount that is directly in the middle of d1 and d2 - 
            similar to z-score of u relative to k based on v * sqrt(t)
        d1 (float): the amount equal to the d1 term in the Black Scholes formula
        d2 (float): the amount equal to the d2 term in the Black Scholes formula

    Results:
        d0 : same as in parameters
        d1 : same as in parameters
        d2 : same as in parameters
        option_value : value of option in same units as underlying value
        delta : change in option_value for a one unit change in underlying value (first order greek)
        vega : change in option_value for a one percent absolute change in volatility (first order greek)
        theta : change in option_value for a one day change in time (first order greek)
        rho : change in option_value for a one percent absolute change in rate (first order greek)
        gamma : change in delta for a one unit change in underlying value (second order greek)
        volga : change in vega for a one percent absolute value change in volatility (second order greek)
        speed : (third order greek)
        ultima : (third order greek)
        vanna : (cross greek)
        charm : (cross greek)
        veta : (cross greek)
        zoma : (cross greek)
        color : (cross greek)
    """
        
    def __init__(self):
        # constructor 
        pass


    # -----------------------
    # helpers
    # -----------------------
    def pv_factor(self, t=1, r=0, **kwargs):
        # Compute time value of money - eventually replace with function from TVM Class
        return np.exp(-r * t) #if interest_rate == 0 then pv_factor will equal one

        
    def to_arrays(self, *args, d_type=float):
        # Convert all inputs to NumPy arrays of dtype=d_type.
        # Raises ValueError if any array contains NaN values.
        # Returns a tuple of arrays in the same order as the inputs.
        arrays = [np.asarray(arg, dtype=d_type) for arg in args]
            
        # Only check NaN for numeric dtypes
        for i, arr in enumerate(arrays):
            if np.issubdtype(arr.dtype, np.number) and np.any(np.isnan(arr)):
                raise ValueError(f"Array {i+1} contains NaN values")
                    
        return tuple(arrays)

    
    def c_or_p(self, c_or_p, array_to_mimic):        
        # Convert array_to_mimic so we can broadcast to its shape if needed
        array_to_mimic, = self.to_arrays(array_to_mimic)
        
        # If scalar indicator, broadcast to array_to_mimic's shape
        if np.isscalar(c_or_p):
            c_or_p = np.full_like(array_to_mimic, c_or_p, dtype=object)

        # Convert to object array and uppercase elementwise
        c_or_p, = self.to_arrays(c_or_p, d_type=object)
        c_or_p = np.char.upper(c_or_p.astype(str))

        call_names = ['C', 'CALL']
        put_names  = ['P', 'PUT']
        valid_names = call_names + put_names
        
        # Validate input
        if not np.all(np.isin(c_or_p, valid_names)):
            raise ValueError("c_or_p must contain only values associated with 'call' or 'put'")

        return np.where(np.isin(c_or_p, call_names), 'CALL', 'PUT')
            
                
    # -----------------------
    # Classic Black-Scholes Formulas
    #      Formulas just calculate answers 
    #      Formulas should be good for use with single inputs or arrays
    #      All data should be checked before using formulas as formulas assume data is consistent and clean
    #      (self, c_or_p='cp', u='u', k='k', v='v', t='t', r='r', d0='d0', d1='d1', d2='d2', diy='diy', **kwargs):
    # -----------------------
        
    def bs_d0_formula(self, u='u', k='k', v='v', t='t', r=0, **kwargs): 
        # Prevent division by zero or invalid sqrt(time)
        t = np.maximum(t, 1e-12)
        v = np.maximum(v, 1e-12)
        vol_sqrt_time = v * np.sqrt(t)  
        log_moneyness = np.log(u / k)
        growth_rate   = r * t         
        return (log_moneyness + growth_rate) / vol_sqrt_time

    
    def bs_d1_formula(self, v='v', t='t', d0='d0', **kwargs):
        return d0 + (v * np.sqrt(t) / 2.0)

    
    def bs_d2_formula(self, v='v', t='t', d0='d0', **kwargs):
        return d0 - (v * np.sqrt(t) / 2.0)

    
    def bs_option_value_formula(self, c_or_p='cp', u='u', k='k', t='t', r=0, d1='d1', d2='d2', **kwargs):
        scalar = np.where(c_or_p == 'CALL', 1, -1)
        u_term  =  scalar * norm.cdf(d1 * scalar) *  u 
        k_term  = -scalar * norm.cdf(d2 * scalar) * (k * self.pv_factor(t=t, r=r))
        return u_term + k_term         
        

    def bs_delta_formula(self, c_or_p='cp', d1='d1', **kwargs):
        call_delta = norm.cdf(d1)
        put_delta  = call_delta - 1  # N(d1) − 1 = −N(−d1)
        return np.where(c_or_p == 'CALL', call_delta, put_delta)

    
    def bs_vega_formula(self, u='u', t='t', d1='d1', **kwargs):
        return u * np.sqrt(t) * norm.pdf(d1) * 0.01

    
    def bs_theta_formula(self, c_or_p='cp', u='u', k='k', v='v', t='t', r=0, d1='d1', d2='d2', diy='diy', **kwargs):
        scalar = np.where(c_or_p == 'CALL', 1, -1)
        u_term  = -1 * u * v * norm.pdf(d1) / (2 * np.sqrt(t))
        k_term  = -1 * k * r * norm.cdf(d2 * scalar) * self.pv_factor(t=t, r=r) * scalar         
        return (u_term + k_term) / diy
        

    def bs_rho_formula(self, c_or_p='cp', k='k', t='t', r=0, d2='d2', **kwargs):
        scalar = np.where(c_or_p == 'CALL', 1, -1)
        return k * t * norm.cdf(d2 * scalar) * self.pv_factor(t=t, r=r) * scalar * 0.01
        

    def bs_gamma_formula(self, u='u', v='v', t='t', d1='d1', **kwargs):
        denominator = u * v * np.sqrt(t)
        return norm.pdf(d1) / denominator  


    def bs_volga_formula(self, v='v', d1='d1', d2='d2', vega='v', **kwargs):
        numerator = vega * d1 * d2
        return numerator / v

    
    # -----------------------
    # Option formulas with data checks
    #      Formulas can be called with multiple input combinations (with or without d-terms, etc.)
    #      Formulas convert inputs to NumPy arrays for elementwise math
    #      Formulas should also work with non-array inputs
    #      (self, c_or_p='cp', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, d1=-100.0, d2=-100.0, diy=-100.0, **kwargs):
    # -----------------------        
        
    def bs_d0(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, **kwargs):
        u, k, t, v, r = self.to_arrays(u, k, t, v, r)
        return self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
        

    def bs_d1(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, **kwargs):
        u, k, t, v, r, d0 = self.to_arrays(u, k, t, v, r, d0)   
        if np.all(d0 == -100.0):
            d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
        return self.bs_d1_formula(d0=d0, v=v, t=t)

    
    def bs_d2(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, **kwargs):
        u, k, t, v, r, d0 = self.to_arrays(u, k, t, v, r, d0)   
        if np.all(d0 == -100.0):
            d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
        return self.bs_d2_formula(d0=d0, v=v, t=t)

       
    def bs_option_value(self, c_or_p='cp', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, 
                        d0=-100.0, d1=-100.0, d2=-100.0, **kwargs):
        u, k, v, t, r, d0, d1, d2 = self.to_arrays(u, k, v, t, r, d0, d1, d2)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, u)
        return self.bs_option_value_formula(c_or_p=c_or_p, u=u, k=k, t=t, r=r, d1=d1, d2=d2)   

    
    # -----------------------
    # greeks (first order)
    # -----------------------
    def bs_delta(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, d1=-100.0,  **kwargs):  
        u, k, v, t, r, d0, d1 = self.to_arrays(u, k, v, t, r, d0, d1)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        c_or_p = self.c_or_p(c_or_p, d1)
        return self.bs_delta_formula(c_or_p=c_or_p, d1=d1)
        

    def bs_vega(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, d1=-100.0,  **kwargs):  
        u, k, v, t, r, d0, d1 = self.to_arrays(u, k, v, t, r, d0, d1)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        return self.bs_vega_formula(u=u, t=t, d1=d1)

    
    def bs_theta(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, 
                 d0=-100.0, d1=-100.0, d2=-100.0, diy=365, **kwargs):
        u, k, v, t, r, d0, d1, d2 = self.to_arrays(u, k, v, t, r, d0, d1, d2)  
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, u)                 
        return self.bs_theta_formula(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, d1=d1, d2=d2, diy=diy)


    def bs_rho(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, d2=-100.0, **kwargs):
        u, k, v, t, r, d0, d2 = self.to_arrays(u, k, v, t, r, d0, d2)  
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, k)       
        return self.bs_rho_formula(c_or_p=c_or_p, k=k, t=t, r=r, d2=d2) 


    def bs_first_order_greeks(self, c_or_p='cp', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, 
                              d0=-100.0, d1=-100.0, d2=-100.0, diy=-100.0, **kwargs):
        u, k, v, t, r, d0, d1, d2, diy = self.to_arrays(u, k, v, t, r, d0, d1, d2, diy) 
        if np.all(d1 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if np.all(d2 == -100.0):
            if np.all(d0 == -100.0):
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)       
        c_or_p = self.c_or_p(c_or_p, u)  
        delta  = self.bs_delta_formula(c_or_p=c_or_p, d1=d1)
        vega   = self.bs_vega_formula(u=u, t=t, d1=d1)
        theta  = self.bs_theta_formula(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, d1=d1, d2=d2, diy=diy)
        rho    = self.bs_rho_formula(c_or_p=c_or_p, k=k, t=t, r=r, d2=d2)
        return (delta, vega, theta, rho)
        
        
    # -----------------------
    # greeks (second order)
    # -----------------------
    def bs_gamma(self, u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, d0=-100.0, d1=-100.0, **kwargs):        
        u, k, v, t, r, d0, d1 = self.to_arrays(u, k, v, t, r, d0, d1)  
        if d1 == [-100]:
            if d0 == [-100.0]:
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)         
        return self.bs_gamma_formula(u=u, v=v, t=t, d1=d1)

        
    def bs_volga(self, c_or_p='', u=-100.0, k=-100.0, v=-100.0, t=-100.0, r=0, 
                 d0=-100.0, d1 = -100.0, d2=-100.0, vega=-100.0, **kwargs):
        u, k, v, t, r, d0, d1, d2 = self.to_arrays(u, k, v, t, r, d0, d1, d2)  
        if d1 == [-100.0]:
            if d0 == [-100.0]:
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d1 = self.bs_d1_formula(d0=d0, v=v, t=t)
        if d2 == [-100]:
            if d0 == [-100.0]:
                d0 = self.bs_d0_formula(u=u, k=k, t=t, v=v, r=r)
            d2 = self.bs_d2_formula(d0=d0, v=v, t=t)     
        if vega == [-100]:
            vega = self.bs_vega_formula(u=u, t=t, d1=d1)
        return self.bs_volga_formula(v=v, d1=d1, d2=d2, vega=vega)


    # -----------------------
    # greeks (third order)
    # -----------------------
    def bs_speed():
        pass
        #pending

    
    def bs_ultima():
        pass
        #pending        


    # -----------------------
    # greeks (cross)
    # -----------------------
    def bs_vanna():
        pass
        #pending


    def bs_charm():
        pass
        #pending


    def bs_veta():
        pass
        #pending


    def bs_zomma():
        pass
        #pending


    def bs_color():
        pass
        #pending


    # -----------------------
    # intrinsic value; extrinsic/time value = option value - intrinsic value
    # -----------------------# 
    def intrinsic_value(self, u='u', pv_k='pv'):
        # Calcs intrinsic values for options
        # Subtract intrinsic values from options prices to get time values
        # Calc the present value of the strike values prior to calling this function
        
        u, pv_k = self.to_arrays(u, pv_k)
        
        call_intrinsic = np.maximum(u - pv_k, 0.0)
        put_intrinsic  = np.maximum(pv_k - u, 0.0)
        
        return call_intrinsic, put_intrinsic
    

In [2]:
u = 100
k = 105
t = 0.5
v = 0.2
r = 0.05

c_or_p = 'CALL'

diy = 365

In [5]:
om = OptionsMath()

In [9]:
d0_1    = om.bs_d0_formula(u=u, k=k, v=v, t=t, r=r)
print(d0_1)
d1_1    = om.bs_d1_formula(v=v, t=t, d0=d0_1)
print(d1_1)
d2_1    = om.bs_d2_formula(v=v, t=t, d0=d0_1)
print(d2_1)
value_1 = om.bs_option_value_formula(c_or_p=c_or_p, u=u, k=k, t=t, r=r, d1=d1_1, d2=d2_1)
print(value_1)
delta_1 = om.bs_delta_formula(c_or_p=c_or_p, d1=d1_1)
print(delta_1)
vega_1  = om.bs_vega_formula(u=u, t=t, d1=d1_1)
print(vega_1)
theta_1 = om.bs_theta_formula(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, d1=d1_1, d2=d2_1, diy=diy)
print(theta_1)
rho_1   = om.bs_rho_formula(c_or_p=c_or_p, k=k, t=t, r=r, d2=d2_1)
print(rho_1)
gamma_1 = om.bs_gamma_formula(u=u, v=v, t=t, d1=d1_1)
print(gamma_1)
volga_1 = om.bs_volga_formula(v=v, d1=d1_1, d2=d2_1, vega=vega_1)
print(volga_1)

-0.1682218640974663
-0.09751118597881155
-0.2389325422161211
4.581680167540007
0.46116022571905085
0.2807568352742075
-0.02107357212521206
0.20767171202182538
0.028075683527420743
0.032706199779395116


In [14]:
d0_2    = om.bs_d0(u=u, k=k, v=v, t=t, r=r)
print(d0_2)
d1_2    = om.bs_d1(v=v, t=t, d0=d0_2)
print(d1_2)
d2_2    = om.bs_d2(v=v, t=t, d0=d0_2)
print(d2_2)
value_2 = om.bs_option_value(c_or_p=c_or_p, u=u, k=k, t=t, r=r, d1=d1_2, d2=d2_2)
print(value_2)
delta_2 = om.bs_delta(c_or_p=c_or_p, d1=d1_2)
print(delta_2)
vega_2  = om.bs_vega(u=u, t=t, d1=d1_2)
print(vega_2)
theta_2 = om.bs_theta(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, d1=d1_2, d2=d2_2, diy=diy)
print(theta_2)
rho_2   = om.bs_rho(c_or_p=c_or_p, k=k, t=t, r=r, d2=d2_2)
print(rho_2)
gamma_2 = om.bs_gamma(u=u, v=v, t=t, d1=d1_2)
print(gamma_2)
volga_2 = om.bs_volga(v=v, d1=d1_2, d2=d2_2, vega=vega_2)
print(volga_2)

-0.1682218640974663
-0.09751118597881155
-0.2389325422161211
4.581680167540007
0.46116022571905085
0.2807568352742075
-0.02107357212521206
0.20767171202182538
0.028075683527420743
0.032706199779395116


In [13]:
d0_3    = om.bs_d0(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(d0_3)
d1_3    = om.bs_d1(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(d1_3)
d2_3    = om.bs_d2(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(d2_3)
value_3 = om.bs_option_value(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(value_3)
delta_3 = om.bs_delta(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(delta_3)
vega_3  = om.bs_vega(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(vega_3)
theta_3 = om.bs_theta(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(theta_3)
rho_3   = om.bs_rho(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(rho_3)
gamma_3 = om.bs_gamma(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r)
print(gamma_3)
volga_3 = om.bs_volga(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, vega=vega_3)
print(volga_3)

-0.1682218640974663
-0.09751118597881155
-0.2389325422161211
4.581680167540007
0.46116022571905085
0.2807568352742075
-0.02107357212521206
0.20767171202182538
0.028075683527420743
0.032706199779395116


In [12]:
all_greeks = om.bs_first_order_greeks(c_or_p=c_or_p, u=u, k=k, v=v, t=t, r=r, diy=diy)
print(all_greeks)

(array(0.46116023), np.float64(0.2807568352742075), np.float64(-0.02107357212521206), np.float64(0.20767171202182538))
